# Choma DB Embedding Testing


In [1]:
from database_manager import MySQLDB,ChromaDBConnector

In [2]:
path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean/data/storage/embeddings'
con = ChromaDBConnector(storage_path=path_store,collection_name='rag_faq2')

# MySQL connection details
CONFIG_SQL_DB = {
    'user': 'root',
    'password': 'admin123',
    'host': 'localhost'
}

DB_NAME = 'data_rag'
TABLE_NAME = 'embedding_table'

sql_db_connector = MySQLDB(CONFIG_SQL_DB,DB_NAME)


/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
Some weights of BertModel were not initialized from the model checkpoint at aari1995/German_Semantic_V3 and are newly initialized: ['embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Collection rag_faq2 already exists with 19038 documents.


In [3]:
ids_retrieve = [f'0.{i}' for i in range(10)]
ids_test = [f'0.{i}' for i in range(10,20)]
data = sql_db_connector.get_records_by_ids(TABLE_NAME,ids_retrieve)

In [4]:
from langchain_core.documents import Document

In [5]:
from sentence_transformers import SentenceTransformer, util
import json
from chromadb.utils import embedding_functions
import chromadb
from sklearn.preprocessing import normalize

def add_documents(col, splits_lists, ids):
    created_ids = []
    for index, x in enumerate(splits_lists):
        current_ids = ids[index]
        col.add(
            ids=current_ids,
            documents=[doc.page_content for doc in x],
            metadatas=[doc.metadata for doc in x]
        )
        created_ids.extend(current_ids)
        # print(f'Actual registers {col.count()}')
    return created_ids

def add_documents_emb(col, splits_lists, ids,embeddings):
    created_ids = []
    for index, x in enumerate(splits_lists):
        current_ids = ids[index]
        current_embs = embeddings[index]
        col.add(
            ids=current_ids,
            documents=[doc.page_content for doc in x],
            metadatas=[doc.metadata for doc in x],
            embeddings = current_embs
        )
        created_ids.extend(current_ids)
        # print(f'Actual registers {col.count()}')
    return created_ids

In [6]:
for i in ['model','embed_model']:
    if i:
        del(i)
# Initialize your model

model = SentenceTransformer("aari1995/German_Semantic_STS_V2",trust_remote_code=True)

# model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True,)
new_col = 'Test_collection2'
client = chromadb.PersistentClient(path=path_store)
client.delete_collection(name=new_col)
embed_model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="aari1995/German_Semantic_STS_V2",normalize_embeddings=True,)

ids = []
docs = []
embeddings = []
for i in data:
    id = i['id']
    chunk = i['content']
    emb = model.encode(chunk,normalize_embeddings=True).tolist()
    metadata = json.loads(i['metadata'])
    doc = Document(page_content=chunk, metadata=metadata)
    ids.append(id)
    docs.append(doc)
    embeddings.append(emb)
    print(f'Doc {doc} id {id}')

col = client.get_or_create_collection(new_col, embedding_function = embed_model)

# add_documents(col,[docs],[ids])
docs_added = add_documents_emb(col,[docs],[ids],[embeddings])
docs_added = add_documents(col,[docs],[ids_test])


No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with mean pooling.
/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with mean pooling.


Doc page_content='Bundesministerium für Wirtschaft und Energie\nRichtlinie für die Bundesförderung für effiziente Gebäude\n– Einzelmaßnahmen (BEG EM)\nVom 17. Dezember 2020\n1 Präambel' metadata={'doc_type': 'Richtlinie BEG EM', 'page': 0, 'type': 'Text', 'source': 'Richtlinie BEG EM (2020-12-17).pdf', 'path': '', 'version': 'X.X', 'valid_date': '17/12/2020', 'id': '0.0', 'pre_id': '', 'post_id': '0.1'} id 0.0
Doc page_content='Mit der Energiewende hat die Bundesrepublik Deutschland eine umfassende und tiefgreifende Transformation ihrer Energieversorgung und Energienutzung eingeleitet. Die Bundesregierung hat sich das Ziel gesetzt, bis 2030 die Treibhausgasemissionen um mindestens 55 Prozent gegenüber dem Basisjahr 1990 zu mindern. Für 2030 gilt, dass der Gebäudebereich nach dem Klimaschutzgesetz (gemäß Quellprinzip) nur noch 70 Millionen Tonnen CO2-Äquivalente emittieren darf. Darüber hinaus hat sich Deutschland das Ziel gesetzt, beim Endenergieverbrauch im Wärme- und Kältesektor, der

In [11]:
len(embeddings[9])

1024

In [13]:
# col.query(query_texts = 'Digitalisierungsmaßnahmen?',n_results=5)
test_data_1 = col.get(ids=ids_retrieve,include=['embeddings','documents'])
test_data_2 = col.get(ids=ids_test,include=['embeddings','documents'])
test_data_3 = con.col.get(ids=ids_retrieve,include=['embeddings','documents'])
# test_data_4 = con.col.get(ids=ids_test,include=['embeddings','documents'])

question = "Gebäudebereich Fortschritte bei der Verringerung"

data_coll = con.col.query(query_texts=question,include=['distances','embeddings','metadatas'])
# data_col1 = col.query(query_texts=question,include=['distances','embeddings'],n_results=20)
# data_col1x = col.query(query_embeddings=model.encode(question,normalize_embeddings=True).tolist(),include=['distances','embeddings'],n_results=20)

# embedding_cdb1 = test_data_1['embeddings']
# embedding_cdb2 = test_data_2['embeddings']
# embedding_cdb3 = test_data_3['embeddings']
# # embedding_cdb4 = test_data_4['embeddings']
# text_cdb = test_data_1['documents']
# text_cdb2 = test_data_3['documents']

# emb_mod_data = model.encode(text_cdb)



BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [64]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

def embed_query_and_compute_similarity(query: str, embeddings_list: list, ids_list: list, model: SentenceTransformer):
    """
    Function to embed a query and compute cosine similarity with a list of precomputed embeddings.
    
    :param query: The query text you want to embed.
    :param embeddings_list: A list of precomputed embeddings (each element should be a vector).
    :param ids_list: A list of document IDs corresponding to the embeddings.
    :param model: A SentenceTransformer model to generate embeddings.
    
    :return: A list of tuples where each tuple contains (document_id, cosine_similarity)
    """
    # Step 1: Embed the query
    query_embedding = model.encode(query)
    
    # Step 2: Normalize the query embedding and precomputed embeddings
    query_embedding_normalized = normalize([query_embedding])[0]
    embeddings_list_normalized = normalize(embeddings_list)
    
    # Step 3: Convert the embeddings to PyTorch tensors for cosine similarity computation
    query_embedding_tensor = torch.tensor(query_embedding_normalized)
    embeddings_tensor = torch.tensor(embeddings_list_normalized)
    
    # Step 4: Compute cosine similarity
    similarities = torch.nn.functional.cosine_similarity(query_embedding_tensor.unsqueeze(0), embeddings_tensor)
    
    # Step 5: Pair each similarity score with its corresponding document ID
    id_similarity_pairs = list(zip(ids_list, similarities.tolist()))
    
    # Sort the results based on similarity scores in descending order (optional)
    id_similarity_pairs.sort(key=lambda x: x[1], reverse=True)
    
    return id_similarity_pairs

# Query
query = "Digitalisierungsmaßnahmen?"
question = "Gebäudebereich Fortschritte bei der Verringerung"

# Get cosine similarities with IDs
cosine_similarities_with_ids_1 = embed_query_and_compute_similarity(question, embedding_cdb1, ids_retrieve, model)

print('\nTest 1 0-9')
# Print results
for doc_id, similarity in cosine_similarities_with_ids_1:
    print(f"Document ID: {doc_id}, Cosine similarity: {similarity}")

# Get cosine similarities with IDs
cosine_similarities_with_ids_2 = embed_query_and_compute_similarity(question, embedding_cdb2, ids_test, model)

print('\nTest 2 11-19')
# Print results
for doc_id, similarity in cosine_similarities_with_ids_2:
    print(f"Document ID: {doc_id}, Cosine similarity: {similarity}")

# Get cosine similarities with IDs
cosine_similarities_with_ids_3 = embed_query_and_compute_similarity(question, embedding_cdb3, ids_retrieve, model)

print('\nOriginal 1 0-9')
# Print results
for doc_id, similarity in cosine_similarities_with_ids_3:
    print(f"Document ID: {doc_id}, Cosine similarity: {similarity}")

# Get cosine similarities with IDs
cosine_similarities_with_ids_4 = embed_query_and_compute_similarity(question, embedding_cdb4, ids_test, model)

print('\nOriginal 2 11-19')
# Print results
for doc_id, similarity in cosine_similarities_with_ids_4:
    print(f"Document ID: {doc_id}, Cosine similarity: {similarity}")


Test 1 0-9
Document ID: 0.2, Cosine similarity: 0.7318269322669463
Document ID: 0.3, Cosine similarity: 0.6712088957797776
Document ID: 0.7, Cosine similarity: 0.6511506446726762
Document ID: 0.1, Cosine similarity: 0.6415166707451161
Document ID: 0.4, Cosine similarity: 0.6115635426072201
Document ID: 0.6, Cosine similarity: 0.6059377253906835
Document ID: 0.9, Cosine similarity: 0.5677447163468046
Document ID: 0.0, Cosine similarity: 0.5388712247728238
Document ID: 0.5, Cosine similarity: 0.5110853817851257
Document ID: 0.8, Cosine similarity: 0.4913829212172147

Test 2 11-19
Document ID: 0.13, Cosine similarity: 0.6992708804491685
Document ID: 0.12, Cosine similarity: 0.6722215410220882
Document ID: 0.17, Cosine similarity: 0.6680760493812912
Document ID: 0.14, Cosine similarity: 0.6312729280584487
Document ID: 0.16, Cosine similarity: 0.6139476252258589
Document ID: 0.11, Cosine similarity: 0.6037422535450536
Document ID: 0.19, Cosine similarity: 0.5577547578931632
Document ID: 0.

In [55]:
embeddings_1 = []
embeddings_2 = []
embeddings_1_ids = []
embeddings_2_ids = []
for n,i in enumerate(data_col1['ids'][0]):
    embeddings_1_ids.append(data_col1['ids'][0][n])
    embeddings_1.append(data_col1['embeddings'][0][n])
    embeddings_2_ids.append(data_col1x['ids'][0][n])
    embeddings_2.append(data_col1x['embeddings'][0][n])
    print(f'{i} - {round(data_col1["distances"][0][n],3)} | {data_col1x['ids'][0][n]} - {round(data_col1x['distances'][0][n],3)}')

0.13 - 0.69 | 0.2 - 0.536
0.2 - 0.71 | 0.13 - 0.601
0.12 - 0.744 | 0.12 - 0.656
0.17 - 0.747 | 0.3 - 0.658
0.1 - 0.75 | 0.17 - 0.664
0.11 - 0.759 | 0.7 - 0.698
0.3 - 0.764 | 0.1 - 0.717
0.14 - 0.818 | 0.14 - 0.737
0.7 - 0.823 | 0.16 - 0.772
0.16 - 0.861 | 0.4 - 0.777
0.4 - 0.885 | 0.6 - 0.788
0.6 - 0.9 | 0.11 - 0.793
0.10 - 0.925 | 0.9 - 0.865
0.19 - 0.959 | 0.19 - 0.884
0.0 - 0.966 | 0.10 - 0.891
0.9 - 0.983 | 0.0 - 0.922
0.15 - 1.08 | 0.15 - 0.972
0.8 - 1.093 | 0.5 - 0.978
0.5 - 1.104 | 0.8 - 1.017
0.18 - 1.116 | 0.18 - 1.046


In [60]:
for n,i in enumerate(embeddings_1):
    
    cos_sim = util.cos_sim(i, embed_question)
    print(f'{embeddings_1_ids[n]} : {len(i)} : {len(embed_question)}')

0.13 : 1024 : 48
0.2 : 1024 : 48
0.12 : 1024 : 48
0.17 : 1024 : 48
0.1 : 1024 : 48
0.11 : 1024 : 48
0.3 : 1024 : 48
0.14 : 1024 : 48
0.7 : 1024 : 48
0.16 : 1024 : 48
0.4 : 1024 : 48
0.6 : 1024 : 48
0.10 : 1024 : 48
0.19 : 1024 : 48
0.0 : 1024 : 48
0.9 : 1024 : 48
0.15 : 1024 : 48
0.8 : 1024 : 48
0.5 : 1024 : 48
0.18 : 1024 : 48


In [21]:
for n,i in enumerate(text_cdb2):
    print(f'Init: {i}')
    print(f'Init2: {text_cdb[n]}')
    print('\n')


Init: Richtlinie BEG EM (2020-12-17)

Bundesministerium für Wirtschaft und Energie
Richtlinie für die Bundesförderung für effiziente Gebäude
– Einzelmaßnahmen (BEG EM)
Vom 17. Dezember 2020
1 Präambel
Init2: Bundesministerium für Wirtschaft und Energie
Richtlinie für die Bundesförderung für effiziente Gebäude
– Einzelmaßnahmen (BEG EM)
Vom 17. Dezember 2020
1 Präambel


Init: Richtlinie BEG EM (2020-12-17)

Mit der Energiewende hat die Bundesrepublik Deutschland eine umfassende und tiefgreifende Transformation ihrer Energieversorgung und Energienutzung eingeleitet. Die Bundesregierung hat sich das Ziel gesetzt, bis 2030 die Treibhausgasemissionen um mindestens 55 Prozent gegenüber dem Basisjahr 1990 zu mindern. Für 2030 gilt, dass der Gebäudebereich nach dem Klimaschutzgesetz (gemäß Quellprinzip) nur noch 70 Millionen Tonnen CO2-Äquivalente emittieren darf. Darüber hinaus hat sich Deutschland das Ziel gesetzt, beim Endenergieverbrauch im Wärme- und Kältesektor, der zu circa zweidrittel

In [18]:
similarity = util.cos_sim(embedding_cdb2, embedding_cdb3)
print(similarity)

tensor([[0.9450, 0.7771, 0.7139, 0.7568, 0.8084, 0.8038, 0.7585, 0.8303, 0.7491,
         0.6927],
        [0.6793, 0.9855, 0.8194, 0.7861, 0.7767, 0.7135, 0.7545, 0.7388, 0.5786,
         0.7032],
        [0.6228, 0.8077, 0.9897, 0.8924, 0.8471, 0.7870, 0.7917, 0.7604, 0.5755,
         0.7849],
        [0.6152, 0.7468, 0.8678, 0.9863, 0.8559, 0.7945, 0.8467, 0.8295, 0.5752,
         0.6732],
        [0.6549, 0.7347, 0.8133, 0.8520, 0.9802, 0.7739, 0.8653, 0.8884, 0.6118,
         0.6967],
        [0.5588, 0.6283, 0.7887, 0.8298, 0.8053, 0.8514, 0.7861, 0.7698, 0.5676,
         0.6692],
        [0.5894, 0.6925, 0.7501, 0.8298, 0.8545, 0.7226, 0.9712, 0.8686, 0.6064,
         0.6512],
        [0.6725, 0.6893, 0.7285, 0.8260, 0.8983, 0.7330, 0.8814, 0.9731, 0.6841,
         0.6709],
        [0.4601, 0.4613, 0.5159, 0.5791, 0.6084, 0.4980, 0.6250, 0.6921, 0.7272,
         0.5719],
        [0.6021, 0.6988, 0.7972, 0.7034, 0.7425, 0.7079, 0.6992, 0.7158, 0.6605,
         0.9755]])


In [12]:
similarity = util.cos_sim(embedding_cdb2, embedding_cdb)
print(similarity)

tensor([[0.9253, 0.6301, 0.6529, 0.6669, 0.7214, 0.6449, 0.6365, 0.6834, 0.5497,
         0.6108],
        [0.6975, 0.8384, 0.7132, 0.7469, 0.7075, 0.6068, 0.6722, 0.6503, 0.4804,
         0.6228],
        [0.6303, 0.7553, 0.8930, 0.8491, 0.7598, 0.7744, 0.7672, 0.7164, 0.5206,
         0.7096],
        [0.6866, 0.7523, 0.7798, 0.9218, 0.8258, 0.8351, 0.8351, 0.7987, 0.6160,
         0.6452],
        [0.7290, 0.7068, 0.7461, 0.7893, 0.9254, 0.7945, 0.8434, 0.8930, 0.6430,
         0.6547],
        [0.6166, 0.6145, 0.7007, 0.7976, 0.7504, 0.9571, 0.7639, 0.7536, 0.6361,
         0.6430],
        [0.6419, 0.6247, 0.6795, 0.7882, 0.7934, 0.7585, 0.9185, 0.8334, 0.6535,
         0.6122],
        [0.7462, 0.6425, 0.7034, 0.7489, 0.8484, 0.7684, 0.8381, 0.9243, 0.7527,
         0.6564],
        [0.5322, 0.4278, 0.5200, 0.4845, 0.5377, 0.5955, 0.5819, 0.6680, 0.9502,
         0.5498],
        [0.6361, 0.6137, 0.7326, 0.6633, 0.6323, 0.6660, 0.6956, 0.6589, 0.5846,
         0.8856]])


In [9]:
model.similarity(embedding_cdb2,embedding_cdb)

tensor([[0.9253, 0.6301, 0.6529, 0.6669, 0.7214, 0.6449, 0.6365, 0.6834, 0.5497,
         0.6108],
        [0.6975, 0.8384, 0.7132, 0.7469, 0.7075, 0.6068, 0.6722, 0.6503, 0.4804,
         0.6228],
        [0.6303, 0.7553, 0.8930, 0.8491, 0.7598, 0.7744, 0.7672, 0.7164, 0.5206,
         0.7096],
        [0.6866, 0.7523, 0.7798, 0.9218, 0.8258, 0.8351, 0.8351, 0.7987, 0.6160,
         0.6452],
        [0.7290, 0.7068, 0.7461, 0.7893, 0.9254, 0.7945, 0.8434, 0.8930, 0.6430,
         0.6547],
        [0.6166, 0.6145, 0.7007, 0.7976, 0.7504, 0.9571, 0.7639, 0.7536, 0.6361,
         0.6430],
        [0.6419, 0.6247, 0.6795, 0.7882, 0.7934, 0.7585, 0.9185, 0.8334, 0.6535,
         0.6122],
        [0.7462, 0.6425, 0.7034, 0.7489, 0.8484, 0.7684, 0.8381, 0.9243, 0.7527,
         0.6564],
        [0.5322, 0.4278, 0.5200, 0.4845, 0.5377, 0.5955, 0.5819, 0.6680, 0.9502,
         0.5498],
        [0.6361, 0.6137, 0.7326, 0.6633, 0.6323, 0.6660, 0.6956, 0.6589, 0.5846,
         0.8856]])

In [11]:
similarity = util.cos_sim(emb_mod_data, embedding_cdb)
print(similarity)

tensor([[1.0000, 0.6323, 0.6283, 0.6692, 0.7442, 0.6421, 0.6298, 0.7005, 0.5770,
         0.6150],
        [0.6323, 1.0000, 0.7452, 0.7883, 0.7217, 0.6031, 0.6625, 0.6317, 0.4525,
         0.5810],
        [0.6283, 0.7452, 1.0000, 0.7805, 0.6986, 0.6983, 0.7025, 0.7014, 0.5211,
         0.6838],
        [0.6692, 0.7883, 0.7805, 1.0000, 0.8052, 0.8083, 0.8453, 0.7515, 0.5255,
         0.6616],
        [0.7442, 0.7217, 0.6986, 0.8052, 1.0000, 0.7762, 0.8247, 0.8902, 0.5816,
         0.6401],
        [0.6421, 0.6031, 0.6983, 0.8083, 0.7762, 1.0000, 0.7757, 0.7809, 0.6553,
         0.6495],
        [0.6298, 0.6625, 0.7025, 0.8453, 0.8247, 0.7757, 1.0000, 0.8496, 0.6011,
         0.6785],
        [0.7005, 0.6317, 0.7014, 0.7515, 0.8902, 0.7809, 0.8496, 1.0000, 0.7169,
         0.6476],
        [0.5770, 0.4525, 0.5211, 0.5255, 0.5816, 0.6553, 0.6011, 0.7169, 1.0000,
         0.5461],
        [0.6150, 0.5810, 0.6838, 0.6616, 0.6401, 0.6495, 0.6785, 0.6476, 0.5461,
         1.0000]])


In [70]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

def compare_chroma_vs_manual(query: str, chroma_collection, ids: list, model: SentenceTransformer):
    """
    Function to compare the cosine similarity calculated by ChromaDB and manually using an embedding model.
    
    :param query: The query text you want to embed and search for.
    :param chroma_collection: The ChromaDB collection to perform the query against.
    :param ids: The list of document IDs corresponding to the embeddings in the collection.
    :param model: A SentenceTransformer model to manually calculate embeddings.
    
    :return: A list of tuples (document_id, chroma_similarity, manual_similarity) for comparison.
    """
    # Step 1: Manually calculate the embedding for the query
    query_embedding_manual = model.encode(query)
    
    # Step 2: Normalize the query embedding
    query_embedding_manual_normalized = normalize([query_embedding_manual])[0]
    
    # Step 3: Perform query on ChromaDB
    chroma_results = chroma_collection.query(query_texts=[query], n_results=len(ids), include=['embeddings','distances'])
    
    print(chroma_results)
    # Extract ChromaDB results (similarity and embeddings)
    chroma_similarity_scores = chroma_results['distances'][0]
    chroma_embeddings = chroma_results['embeddings'][0]
    
    # Step 4: Manually calculate cosine similarity for each document
    chroma_embeddings_normalized = normalize(chroma_embeddings)
    
    query_embedding_tensor = torch.tensor(query_embedding_manual_normalized)
    chroma_embeddings_tensor = torch.tensor(chroma_embeddings_normalized)
    
    manual_similarities = torch.nn.functional.cosine_similarity(
        query_embedding_tensor.unsqueeze(0), chroma_embeddings_tensor
    ).tolist()
    
    # Step 5: Return a comparison between ChromaDB's similarity and manual similarity
    comparison = []
    for i, doc_id in enumerate(ids):
        comparison.append((doc_id, chroma_similarity_scores[i], manual_similarities[i]))
    
    return comparison

# Example usage
# Load your SentenceTransformer model (should match ChromaDB's embedding function)

# Example query and document IDs

# Get the ChromaDB collection (assuming it has already been set up)
# col = your_chroma_db_collection_instance

# Compare ChromaDB cosine similarity vs manual similarity
results = compare_chroma_vs_manual(question, col, ids_test, model)

# Print results
for doc_id, chroma_similarity, manual_similarity in results:
    print(f"Document ID: {doc_id}, ChromaDB Cosine Similarity: {chroma_similarity}, Manual Cosine Similarity: {manual_similarity}")

{'ids': [['0.13', '0.2', '0.12', '0.17', '0.1', '0.11', '0.3', '0.14', '0.7', '0.16']], 'distances': [[0.6900288174301313, 0.7098977257682157, 0.7440621665914056, 0.7471734604341227, 0.749936079719571, 0.7593593224942761, 0.7641178133749978, 0.8176705599870844, 0.8225732720485194, 0.8610068049022108]], 'metadatas': None, 'embeddings': [[[0.08977916091680527, 0.025256583467125893, 0.005022294819355011, 0.004833047743886709, 0.014540941454470158, 0.020335262641310692, 0.01092371717095375, 0.03005397692322731, -0.007371747400611639, 0.028594166040420532, 0.043057601898908615, -0.04579118266701698, 0.05710102617740631, -0.007727408315986395, 0.02893763594329357, 0.05987313762307167, -0.047527432441711426, -0.018626360222697258, 0.004179168492555618, -0.010312256403267384, -0.03325752541422844, -0.042205773293972015, -0.09732893109321594, -0.1007152572274208, 0.017344020307064056, 0.034287143498659134, 0.04187527298927307, -0.01716410182416439, -0.08114778995513916, -0.020228488370776176, 0